In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
import torch.optim as optim

# Load Data

In [22]:
class MNISTDataset(Dataset):
    def __init__(self, datapath="./data/", partition="train", transform=None):
        if(partition=="test"):
           self.dataframe = pd.read_csv(datapath+"mnist_test.csv")
        else:
            self.dataframe = pd.read_csv(datapath+"mnist_train.csv")
        self.transform = transform
        self.images = self.dataframe.drop('label', axis=1).values.astype(np.float32)
        self.labels = self.dataframe['label'].values.astype(np.int64)

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        image = self.images[index]
        label = self.labels[index]
        # Convert the image to the PyTorch tensor format ( H, W, C)
        image = image.view().reshape(28,28,1)
        label = label
        if self.transform:
            image = self.transform(image)
        return image, label

train= MNISTDataset(transform=ToTensor())
test = MNISTDataset(partition="test",transform=ToTensor())
x,y = train[0]
print(x.shape,y)

torch.Size([1, 28, 28]) 5


In [23]:
class CNN(nn.Module):
    def __init__(self,out_channel=10):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(32 * 7 * 7, 120)
        self.fc3 = nn.Linear(120, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x))) #torch.Size([1, 16, 14, 14])
        x = self.pool(F.relu(self.conv2(x))) #torch.Size([1, 32, 7, 7])
        x = torch.flatten(x, 1) # flatten all dimensions except batch torch.Size([1, 1568])
        x = F.relu(self.fc1(x))
        x = self.fc3(x)
        return x


data = torch.rand((1,1,28,28)) # batch size, dim, h, w
cnn = CNN()
cnn(data)

tensor([[ 0.0138, -0.0039,  0.0009, -0.0765,  0.0405, -0.1190, -0.0782, -0.1302,
         -0.0595, -0.0659]], grad_fn=<AddmmBackward0>)

### **train**

In [24]:
trainloader = DataLoader(train, batch_size=64, shuffle=True)
testloader = DataLoader(test, batch_size=64, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
model.train()
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch + 1}/{num_epochs}] Loss: {running_loss / len(trainloader)}")

Epoch [1/10] Loss: 0.20499537247225647
Epoch [2/10] Loss: 0.05060625662118681
Epoch [3/10] Loss: 0.03782531623422021
Epoch [4/10] Loss: 0.031980503450916915
Epoch [5/10] Loss: 0.026405449566383375
Epoch [6/10] Loss: 0.024696606560883665
Epoch [7/10] Loss: 0.02252572300607802
Epoch [8/10] Loss: 0.019069363040259222
Epoch [9/10] Loss: 0.017325226723808405
Epoch [10/10] Loss: 0.01456247013881644


### **evaluation**

In [25]:
#test
model.eval()
with torch.no_grad():
        correct = 0
        total = 0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outs = model(inputs)
            pred_y = torch.max(outs, 1)[1].data.squeeze()
            accuracy = (pred_y == labels).sum().item() / float(labels.size(0))

            
print(f'Test Accuracy of the model on the {len(trainloader.dataset)} training images: %.2f' % accuracy)

#test
model.eval()
with torch.no_grad():
        correct = 0
        total = 0
        for i, data in enumerate(testloader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outs = model(inputs)
            pred_y = torch.max(outs, 1)[1].data.squeeze()
            accuracy = (pred_y == labels).sum().item() / float(labels.size(0))

            
print(f'Test Accuracy of the model on the {len(testloader.dataset)} test images: %.2f' % accuracy)


Test Accuracy of the model on the 60000 training images: 1.00
Test Accuracy of the model on the 10000 test images: 1.00
